# Project 2  AIDI 1005  - Aman Joshi 200543257 

## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [1]:
# Importing the neccessary libraries in the notebook

!pip install cassandra-driver
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

In [2]:
# Import Cassandra

import cassandra

# Import Cluster from cassandra

from cassandra.cluster import Cluster 

#### Creating a Cluster

In [3]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) with port 6000 and 
# create a session to establish connection and begin executing queries

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'], port = 6000)
    session = cluster.connect()
except Exception as e:
    print(e)    

In [4]:
# !!!Only run if you want to delete keyspace!!! (if you re-run the code)

session.execute("DROP KEYSPACE soundcloud")

#### Create & Set Keyspace

In [5]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above

keyspace = "soundcloud"

query = "SELECT keyspace_name FROM system_schema.keyspaces WHERE keyspace_name='{}'".format(keyspace)

result = session.execute(query)

session.execute("CREATE KEYSPACE soundcloud WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};")

session.set_keyspace(keyspace)

## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




-----------------------------------

### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [6]:
# Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
# that was heard during session_number = 338, and item_in_session_number = 4
# make use of create table command     

session.execute("CREATE TABLE SoundCloud1 (user_id int, artist_name text,song_title text,session_number int,item_in_session_number int,length float, PRIMARY KEY((session_number, item_in_session_number), user_id,artist_name)) WITH CLUSTERING ORDER BY (user_id ASC);")

### Let's insert our data into of table

In [7]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
# Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO SoundCloud1 (user_id, artist_name, song_title, session_number, item_in_session_number, length) VALUES (%s, %s, %s,%s, %s, %s)"
        
        # Task: Match the column in the csv file to the column in the INSERT statement.
        # e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        # e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[10]), row[0], row[9], int(row[8]), int(row[3]), float(row[5])))    

### Validate our Data Model using a SELECT

In [8]:
# Task: Make use of the SELECT statement and for loop to check if your query works and display the results
# Execute the query

query = "SELECT * FROM SoundCloud1 WHERE session_number = 338 AND item_in_session_number = 4 ALLOW FILTERING;"
rows = session.execute(query)
for row in rows:
    print(row.artist_name,row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


----------------------

### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [9]:
# Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
# name(fname and lname) of the user for user_id = 10, session_number = 182 
# make use of create table command

session.execute("CREATE TABLE SoundCloud2 (user_id int,fname text, lname text,artist_name text,song_title text,session_number int,item_in_session_number int, PRIMARY KEY((artist_name,song_title) ,item_in_session_number)) WITH CLUSTERING ORDER BY ( item_in_session_number DESC);")

### Let's insert our data into of table

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below

file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
# Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO SoundCloud2 (user_id, fname, lname, artist_name, song_title, session_number, item_in_session_number) VALUES (%s, %s, %s,%s, %s, %s,%s)"
        
        # Task: Match the column in the csv file to the column in the INSERT statement.
        # e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        # e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[10]), row[1], row[4], row[0], row[9], int(row[8]), int(row[3])))

### Validate our Data Model using a SELECT

In [11]:
# Task: Make use of the SELECT statement and for loop to check if your query works and display the results

rows = session.execute("SELECT * FROM SoundCloud2 WHERE user_id = 10 AND session_number = 182 ALLOW FILTERING;")
for row in rows:
    print(row.item_in_session_number,row.artist_name,row.song_title, row.fname, row.lname)

2 Sebastien Tellier Kilometer Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
0 Down To The Bone Keep On Keepin' On Sylvie Cruz


-------------------------

### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [12]:
# Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
# to the song_title 'All Hands Against His Own'
# make use of create table command

session.execute("CREATE TABLE SoundCloud3 (fname text, lname text,song_title text, PRIMARY KEY((song_title)));")

### Let's insert our data into of table

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
# Task: Write the INSERT statements and assign it to the query variable

        query = "INSERT INTO SoundCloud3 (fname, lname, song_title) VALUES (%s, %s, %s)"
        
        # Task: Match the column in the csv file to the column in the INSERT statement.
        # e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        # e.g., if you want to INSERT location from csv file into database you will use row[7]
        
        session.execute(query, (row[1], row[4], row[9]))

### Validate our Data Model using a SELECT

In [14]:
# Task: Make use of the SELECT statement and for loop to check if your query works and display the results

rows = session.execute("SELECT * FROM SoundCloud3 WHERE song_title='All Hands Against His Own' ALLOW FILTERING;")
for row in rows:
    print(row.fname, row.lname, row.song_title)

Sara Johnson All Hands Against His Own


---------------------------------

### Drop the tables before closing out the sessions

In [15]:
# Dropping and closing the session

session.execute("DROP TABLE soundcloud.SoundCloud1")
session.execute("DROP TABLE soundcloud.SoundCloud2")
session.execute("DROP TABLE soundcloud.SoundCloud3")

### Close the session and cluster connection¶

In [16]:
# End the session and disconnect the cluster connection

session.shutdown()
cluster.shutdown()

-----------